In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
dataset = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')

# Imputing Missing values

In [3]:
missing_col = ['imbalance_size', 'matched_size', 'reference_price', 'far_price', 'near_price',
               'bid_price', 'ask_price', 'wap', 'target']
dataset[missing_col] = dataset.groupby(['date_id','stock_id'])[missing_col].transform(lambda x: x.fillna(x.median()))
dataset[missing_col] = dataset.groupby(['stock_id'])[missing_col].transform(lambda x: x.fillna(x.median()))

In [5]:
df = dataset

# Creating a the missing records and therefore a balanced dataset 

In [6]:

stock_counts = df['stock_id'].value_counts()
stock_ids_to_process = stock_counts[stock_counts < 26455].index.tolist()
stock_ids_to_process.sort()
unique_time_ids = df['time_id'].unique().tolist()
# Calculating median values for each feature over groups of 'stock_id'
grouped_medians = df.groupby('stock_id').median()

# Determined desired count for each 'time_id' within a group
desired_count_per_group = 26455 #(55x481 per stock)

# Create a new DataFrame to store the missing data
missing_data = pd.DataFrame(columns=df.columns)

# Counter for tracking progress
counter = 0

for stock_id in stock_ids_to_process:
    # Filter rows for the current 'stock_id'
    stock_subset = df[df['stock_id'] == stock_id]
    present = stock_subset['time_id'].unique().tolist()
    un_present = list(set(unique_time_ids) ^ set(present))
    # Iterate over unique 'time_id' values within the current 'stock_id'
    for time_id in un_present:
        # Filter rows for the current 'time_id' within the current 'stock_id'
        # Create a DataFrame with missing values for other features
        missing_rows = pd.DataFrame(np.nan, index=np.arange(1), columns=df.columns)

        # Fill each feature with median values for the current 'stock_id'
        for feature in df.columns:
            if feature not in ['stock_id', 'time_id', 'seconds_in_bucket', 'date_id']:
                missing_rows[feature] = stock_subset[feature].median()  # Fix the typo here

        # Set 'stock_id' and 'time_id' to the current values
        missing_rows['stock_id'] = stock_id
        missing_rows['time_id'] = time_id
        missing_rows['seconds_in_bucket'] = (time_id % 55) * 10
        missing_rows['date_id'] = time_id // 55
        # Append to the balanced data DataFrame
        missing_data = pd.concat([missing_data, missing_rows], ignore_index=True)
        
    # Print progress
    print(f"Processed stock_id {stock_id} ({counter + 1} of {len(stock_ids_to_process)})")
    counter += 1


Processed stock_id 69 (1 of 11)
Processed stock_id 73 (2 of 11)
Processed stock_id 78 (3 of 11)
Processed stock_id 79 (4 of 11)
Processed stock_id 99 (5 of 11)
Processed stock_id 102 (6 of 11)
Processed stock_id 135 (7 of 11)
Processed stock_id 150 (8 of 11)
Processed stock_id 153 (9 of 11)
Processed stock_id 156 (10 of 11)
Processed stock_id 199 (11 of 11)


In [8]:
balanced_data = pd.concat([dataset,missing_data])

In [16]:
balanced_data = balanced_data.sort_values(by=['date_id', 'seconds_in_bucket','stock_id'])

In [17]:
balanced_data.reset_index(drop=True)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1.0,0.999812,13380276.64,0.999812,0.999812,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1.0,0.999896,1642214.25,0.991141,0.999661,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1.0,0.999561,1819368.03,0.999772,0.999772,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1.0,1.000171,18389745.62,1.002021,1.001590,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1.0,0.999532,17860614.95,1.007894,1.007894,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5290995,195,480,540,2440722.89,-1.0,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5290996,196,480,540,349510.47,-1.0,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5290997,197,480,540,0.00,0.0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5290998,198,480,540,1000898.84,1.0,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [18]:
balanced_data.to_csv('balanced_data.csv',index = False)